In [9]:
import tkinter as tk
from tkinter import filedialog, Label
from keras.models import load_model
from PIL import Image, ImageTk
import numpy as np
import cv2

model = load_model('C:/Users/thano/Downloads/data science/interm/project 3/newsign_language_model.h5')



# function for image detection

In [10]:

def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        process_image(file_path)
        
def process_image(file_path):
    
    image = Image.open(file_path).convert('RGB')
    image = image.resize((64, 64))
    image_array = np.array(image) / 255.0  # Normalize

    image_array = np.expand_dims(image_array, axis=0)

    prediction = model.predict(image_array)
    predicted_index = np.argmax(prediction, axis=1)[0]
    predicted_class_name = class_names[predicted_index]  
    
    result_label.config(text=f"Predicted Sign: {predicted_class_name}")


    img = ImageTk.PhotoImage(image.resize((200, 200)))
    image_label.config(image=img)
    image_label.image = img
    



# function for real time viedoe

In [11]:
box_start_x = 200  
box_start_y = 150  
box_width = 200    
box_height = 200  


def start_video():
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        cv2.rectangle(frame, (box_start_x, box_start_y), 
                      (box_start_x + box_width, box_start_y + box_height), 
                      (255, 0, 0), 2) 


        roi = frame[box_start_y:box_start_y + box_height, box_start_x:box_start_x + box_width]
        img = cv2.resize(roi, (64, 64))  
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
        img_array = np.expand_dims(np.array(img) / 255.0, axis=0) 
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction, axis=1)
        cv2.putText(frame, f"Predicted Sign: {predicted_class[0]}", (10, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
        cv2.imshow('Real-Time Sign Language Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows() 

# giu (to stop video click q)

In [12]:

root = tk.Tk()
root.title("Sign Language Detection")
root.geometry("800x600")
root.config(bg='lightblue') 


upload_button = tk.Button(root, text="Upload Image", command=upload_image, width=30, height=5,bg='lightpink',fg='black')
upload_button.pack(pady=20)

video_button = tk.Button(root, text="Start Video Detection", command=start_video, width=30, height=5,bg='lightpink',fg='black')
video_button.pack(pady=20)

result_label = Label(root, text="Predicted Sign: None", font=("Helvetica", 16))
result_label.pack(pady=20)

image_label = Label(root)
image_label.pack(pady=10)


root.mainloop()